# Tuning an SVC Model

copied from `xgb_fitting.ipynb`

The goal of this notebook is to train and evaluate an SVC model, comparing it's performance on a holdout set against other types of models (LR,LDA, XGBoost). 

To ensure reproducibility and consistent evaluation across models, all datasets were **pre-split into cross-val data and holdout data** as below:

| Split type           | CV training file     | Holdout file              | Description                              |
| -------------------- | -------------------- | ------------------------- | ---------------------------------------- |
| **Random**           | `apps_cv_random.csv` | `apps_holdout_random.csv` | Simple random sampling                   |
| **Stratified**       | `apps_cv_strat.csv`  | `apps_holdout_strat.csv`  | Stratified by `TARGET`                   |
| **Multi-Stratified** | `apps_cv_multi.csv`  | `apps_holdout_multi.csv`  | Stratified by `TARGET` + `CODE_GENDER_M` |

Each dataset for cross-validation (`apps_cv_*.csv`) also contains a column, `fold`, with pre-assigned folds from 1-5 using the corresponding splitting method to ensure consistent evaluation. Therefore, no additional splitting is needed inside this notebook -- can simply loop through assigned folds for cross-validation.


In [2]:
import pandas as pd 
import numpy as np 
from sklearn.svm import SVC
from sklearn.decomposition import PCA

Copied from `XGB_fitting.ipynb`

In [4]:
# METRICS 

def classification_metrics(y_true, y_pred):
    """
    Computes confusion matrix + accuracy, precision, recall, F1, and balanced accuracy.
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    # Confusion matrix components
    tp = np.sum((y_true == 1) & (y_pred == 1))
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))

    # Metrics
    acc  = (tp + tn) / max((tp + tn + fp + fn), 1)
    prec = tp / max((tp + fp), 1)
    rec  = tp / max((tp + fn), 1)
    f1   = (2 * prec * rec / max((prec + rec), 1e-12)) if (prec + rec) > 0 else 0.0

    # Specificity (True Negative Rate)
    spec = tn / max((tn + fp), 1)

    # Balanced accuracy
    bal_acc = 0.5 * (rec + spec)

    metrics = {
        "n": len(y_true),
        "tp": tp, "tn": tn, "fp": fp, "fn": fn,
        "acc": acc, "bal_acc": bal_acc, "prec": prec, "rec": rec, "spec": spec,
        "f1": f1
    }
    return metrics

def roc_auc_from_probs(y_true, y_prob):
    
    desc_sort_indices = np.argsort(-y_prob)
    y_true = np.array(y_true)[desc_sort_indices]
    y_prob = np.array(y_prob)[desc_sort_indices]
    pos = np.sum(y_true == 1)
    neg = np.sum(y_true == 0)

    # running totals for TPR/FPR
    tpr = [0.0]
    fpr = [0.0]
    tp = fp = 0
    for i in range(len(y_true)):
        if y_true[i] == 1:
            tp += 1
        else:
            fp += 1
        tpr.append(tp / pos)
        fpr.append(fp / neg)

    # get auc
    auc = np.trapz(tpr, fpr)
    return auc

def pr_auc_from_probs(y_true, y_prob):
    # Sort by predicted probability descending
    desc_sort_indices = np.argsort(-y_prob)
    y_true = np.array(y_true)[desc_sort_indices]
    y_prob = np.array(y_prob)[desc_sort_indices]
    
    tp = 0
    fp = 0
    pos = np.sum(y_true == 1)
    
    precision = [1.0]  # starts at 1 when recall=0
    recall = [0.0]
    
    for i in range(len(y_true)):
        if y_true[i] == 1:
            tp += 1
        else:
            fp += 1
        prec = tp / (tp + fp)
        rec = tp / pos
        precision.append(prec)
        recall.append(rec)
    
    # ensure it ends at recall=1
    precision = np.array(precision)
    recall = np.array(recall)
    
    # integrate area under curve
    auc_pr = np.trapz(precision, recall)
    return auc_pr

In [5]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

def cv_svc_pca(data, feature_cols, target_col, params=None, n_components=0.95):
    
    if params is None:
        params = {}
    params = {
        "dual": True,
        "class_weight": "balanced",
        **params
    }

    fold_metrics = []

    for f in sorted(data.fold.unique()):
        train = data[data.fold != f]
        test  = data[data.fold == f]
        X_train, y_train = train[feature_cols], train[target_col]
        X_test,  y_test  = test[feature_cols],  test[target_col]

        pipe = make_pipeline(
            StandardScaler(),
            PCA(n_components=n_components),
            LinearSVC(**params)
        )

        pipe.fit(X_train, y_train)

        y_pred = pipe.predict(X_test)
        y_scores = pipe.decision_function(X_test)
        y_train_scores = pipe.decision_function(X_train)
        # y_prob = pipe.predict_proba(X_test)
        # y_train_prob = pipe.predict_proba(X_train)

        m = classification_metrics(y_test, y_pred)
        m["roc_auc"]       = roc_auc_from_probs(y_test, y_scores)
        m["train_roc_auc"] = roc_auc_from_probs(y_train, y_train_scores)
        m["fold"] = int(f)
        fold_metrics.append(m)

    return pd.DataFrame(fold_metrics).sort_values("fold").reset_index(drop=True)

# Model Development

**Notes:** 
- All evaluation will focus on stratified cross-validation, but we will test the other methods as well. 
- Recall that folds have been pre-assigned to ensure consistency across different model development processes
- We have decided to scale + PCA.

**Process:**
1. Setting a baseline
    - evaluating an xgb model with all default parameters to build off of
2. Hyperparameter tuning
    - evaluate many different combinations of parameters
    - choose the best set based on average ROC-AUC across all folds
3. Holdout evaluation
    - evaluate on the corresponding holdout table. the performance here is what we will compare with other models (LR, SVC, LDA)
4. Threshold tuning
    - tweak the threshold on the best model to maximize another chosen metric (recall, precision, f1, balanced accuracy, etc.) 
        - note that roc-auc is not affected by threshold, hence the need a different optimizing metric
    - what metric we choose to optimize with threshold depends on business needs
        - consider the cost of mislabeling someone as high risk? or trusting an applicant that you shouldn't? will there be human review?
        - something we can include in the right up as optionality moving forward, not something we have to decide now on our own
        - "our model is very solid at ranking applicants from low-risk to high-risk, but in terms of actual classification, we can move the threshold based on what matters most to the business"

# setup

In [4]:
apps_cv_strat = pd.read_csv("data/apps_cv_strat.csv")
apps_holdout_strat = pd.read_csv("data/apps_holdout_strat.csv")
target_col = 'TARGET'
feature_cols = [col for col in apps_cv_strat.columns if col not in 
                [target_col, 'SK_ID_CURR', 'fold', 'neighbors_target_mean_500' , 'AGE_INT', 'DAYS_BIRTH', 'CODE_GENDER_M', 'CODE_GENDER_XNA', 'NAME_FAMILY_STATUS_Previously Married', 'NAME_FAMILY_STATUS_Single']]
print(sorted(feature_cols))

['AMT_ANNUITY', 'AMT_CREDIT', 'AMT_GOODS_PRICE', 'AMT_INCOME_TOTAL', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'BUREAU_REQ_MISSING', 'BUREAU_REQ_TOTAL', 'CNT_CHILDREN', 'CNT_FAM_MEMBERS', 'DAYS_CREDIT_mean_external', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH', 'DAYS_LAST_PHONE_CHANGE', 'DAYS_REGISTRATION', 'DEF_30_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'EMERGENCYSTATE_MODE_UNKNOWN', 'EMERGENCYSTATE_MODE_Yes', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'EXT_SOURCE_MEAN', 'FLAG_CONT_MOBILE', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCU

In [10]:
params = {}
results = cv_svc_pca(apps_cv_strat, feature_cols, target_col, params=params)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [11]:
results

,n,tp,tn,fp,fn,acc,bal_acc,prec,rec,spec,f1,roc_auc,train_roc_auc,fold
0,49156,1662,39535,5652,2307,0.838087,0.646833,0.227235,0.418745,0.874920,0.294602,0.734349,0.734456,1
1,49156,1987,37169,8018,1982,0.796566,0.661595,0.198601,0.500630,0.822560,0.284385,0.739607,0.735101,2
2,49156,1839,37662,7525,2130,0.803585,0.648405,0.196390,0.463341,0.833470,0.275857,0.725445,0.740305,3
3,49155,1994,37395,7791,1975,0.801322,0.664986,0.203781,0.502394,0.827579,0.289952,0.743978,0.741105,4
4,49154,1935,37294,7892,2033,0.798084,0.656498,0.196906,0.487651,0.825344,0.280536,0.734690,0.736279,5


In [8]:
from itertools import product
import time

def grid_search_SVC(data, feature_cols, target_col, param_grid):
    
    # get all possible combinations of parameters
    keys = list(param_grid.keys())
    combos = [dict(zip(keys, v)) for v in product(*param_grid.values())]

    # initialize stuff for tracking and results
    results = []
    total = len(combos)
    start = time.time()
    next_checkpoint = 5 
    best_roc_auc = 0
    best_params = None

    # evaluate every possible combo
    for i, params in enumerate(combos, 1):

        # run cross validation and store results
        fold_results = cv_svc_pca(data, feature_cols, target_col, params)
        mean_roc_auc = fold_results["roc_auc"].mean()

        results.append({
            'params': params,
            'mean_roc_auc': mean_roc_auc,
            'mean_f1': fold_results['f1'].mean(),
            'mean_acc': fold_results['acc'].mean(),
            'mean_bal_acc': fold_results['bal_acc'].mean(),
            'mean_prec': fold_results['prec'].mean(),
            'mean_rec': fold_results['rec'].mean(),
        })

        # tracker for updates
        if  mean_roc_auc > best_roc_auc:
            best_roc_auc =  mean_roc_auc
            best_params = params

        # print progress checkpoints
        pct_done = (i/total)*100
        elapsed = time.time() - start
        if pct_done >= next_checkpoint or i == total:
            print(f"{i}/{total} ({pct_done:5.1f}% in {elapsed/60:.1f} mins) | Best ROC-AUC: {best_roc_auc:.4f} | Best Params: {best_params}")
            next_checkpoint += 5


    
    return pd.DataFrame(results).sort_values("mean_roc_auc", ascending=False).reset_index(drop=True)

In [16]:
param_grid = {
    "C": [0.001, 0.01, 0.1, 1, 10, 100],
    "loss": ["hinge", "squared_hinge"]
}

wide_search_results = grid_search_SVC(apps_cv_strat, feature_cols, target_col, param_grid)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


1/12 (  8.3% in 1.0 mins) | Best ROC-AUC: 0.7533 | Best Params: {'C': 0.001, 'loss': 'hinge'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


2/12 ( 16.7% in 8.4 mins) | Best ROC-AUC: 0.7533 | Best Params: {'C': 0.001, 'loss': 'hinge'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


...

10/12 ( 83.3% in 59.1 mins) | Best ROC-AUC: 0.7535 | Best Params: {'C': 0.1, 'loss': 'hinge'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


11/12 ( 91.7% in 66.5 mins) | Best ROC-AUC: 0.7535 | Best Params: {'C': 0.1, 'loss': 'hinge'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


12/12 (100.0% in 73.8 mins) | Best ROC-AUC: 0.7535 | Best Params: {'C': 0.1, 'loss': 'hinge'}


In [17]:
wide_search_results

,params,mean_roc_auc,mean_f1,mean_acc,mean_bal_acc,mean_prec,mean_rec
0,"{'C': 0.1, 'loss': 'hinge'}",0.753527,0.265728,0.698556,0.688072,0.165395,0.675570
1,"{'C': 0.1, 'loss': 'squared_hinge'}",0.753480,0.266677,0.701555,0.688140,0.166339,0.672143
2,"{'C': 0.01, 'loss': 'hinge'}",0.753447,0.265690,0.698723,0.687933,0.165396,0.675066
3,"{'C': 0.01, 'loss': 'squared_hinge'}",0.753357,0.265717,0.697917,0.688368,0.165301,0.676981
4,"{'C': 0.001, 'loss': 'hinge'}",0.753259,0.265232,0.697226,0.687923,0.164936,0.676830
5,"{'C': 0.001, 'loss': 'squared_hinge'}",0.753234,0.265612,0.697551,0.688376,0.165193,0.677434
6,"{'C': 1, 'loss': 'hinge'}",0.745827,0.266079,0.707475,0.684305,0.166877,0.656673
7,"{'C': 1, 'loss': 'squared_hinge'}",0.731704,0.280578,0.801096,0.653681,0.199121,0.477878
8,"{'C': 10, 'loss': 'squared_hinge'}",0.606105,0.124352,0.867762,0.531312,0.138993,0.130068
9,"{'C': 100, 'loss': 'squared_hinge'}",0.589194,0.108899,0.876880,0.521904,0.141792,0.098568


In [22]:
powers = np.arange(-3, -1, 0.1)
grid_values = [10**x for x in powers]

param_grid = {
    "C": grid_values,
    "loss": ["hinge"]
}

search_results2 = grid_search_SVC(apps_cv_strat, feature_cols, target_col, param_grid)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


1/20 (  5.0% in 1.0 mins) | Best ROC-AUC: 0.7533 | Best Params: {'C': 0.001, 'loss': 'hinge'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


2/20 ( 10.0% in 2.0 mins) | Best ROC-AUC: 0.7533 | Best Params: {'C': 0.0012589254117941675, 'loss': 'hinge'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


...

18/20 ( 90.0% in 44.2 mins) | Best ROC-AUC: 0.7536 | Best Params: {'C': 0.03162277660168389, 'loss': 'hinge'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


19/20 ( 95.0% in 49.4 mins) | Best ROC-AUC: 0.7536 | Best Params: {'C': 0.03162277660168389, 'loss': 'hinge'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


20/20 (100.0% in 54.9 mins) | Best ROC-AUC: 0.7536 | Best Params: {'C': 0.03162277660168389, 'loss': 'hinge'}


In [23]:
search_results2

,params,mean_roc_auc,mean_f1,mean_acc,mean_bal_acc,mean_prec,mean_rec
0,"{'C': 0.03162277660168389, 'loss': 'hinge'}",0.753587,0.265601,0.698759,0.687769,0.165351,0.674663
1,"{'C': 0.01995262314968885, 'loss': 'hinge'}",0.753518,0.265682,0.698731,0.687915,0.165393,0.675015
2,"{'C': 0.03981071705534985, 'loss': 'hinge'}",0.753513,0.265819,0.698943,0.688030,0.165499,0.675015
3,"{'C': 0.015848931924611172, 'loss': 'hinge'}",0.753502,0.265716,0.698808,0.687934,0.165422,0.674965
4,"{'C': 0.025118864315095874, 'loss': 'hinge'}",0.753499,0.265712,0.698869,0.687898,0.165428,0.674814
5,"{'C': 0.06309573444801955, 'loss': 'hinge'}",0.753489,0.265758,0.698654,0.688079,0.165424,0.675469
6,"{'C': 0.012589254117941701, 'loss': 'hinge'}",0.753484,0.265838,0.698861,0.688100,0.165499,0.675267
7,"{'C': 0.07943282347242846, 'loss': 'hinge'}",0.753480,0.265624,0.698918,0.687741,0.165386,0.674411
8,"{'C': 0.010000000000000021, 'loss': 'hinge'}",0.753473,0.265706,0.698678,0.687978,0.165399,0.675217
9,"{'C': 0.0501187233627274, 'loss': 'hinge'}",0.753434,0.265550,0.698711,0.687720,0.165315,0.674612


We found the best value of `C` to be 10^-1.5

# Holdout Evaluation

In [6]:
# make train and test sets
train = apps_cv_strat
test = apps_holdout_strat
X_train, y_train = train[feature_cols], train[target_col]
X_test,  y_test  = test[feature_cols],  test[target_col]

# set up pipeline with scaling, PCA, and SVC
pipe = make_pipeline(
    StandardScaler(),
    PCA(n_components=0.95),
    LinearSVC(C=10**-1.5, loss="hinge", dual=True, class_weight="balanced")
)

# fit and predict
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
y_scores = pipe.decision_function(X_test)
y_train_scores = pipe.decision_function(X_train)

# get metrics
m = classification_metrics(y_test, y_pred)
m["roc_auc"]       = roc_auc_from_probs(y_test, y_scores)
m["train_roc_auc"] = roc_auc_from_probs(y_train, y_train_scores)
m["pr_roc_auc"] = pr_auc_from_probs(y_test, y_scores)
m

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


{'n': 61443,
 'tp': 3443,
 'tn': 39794,
 'fp': 16689,
 'fn': 1517,
 'acc': 0.7036928535390524,
 'bal_acc': 0.6993418962628208,
 'prec': 0.17102125968607193,
 'rec': 0.6941532258064517,
 'spec': 0.7045305667191899,
 'f1': 0.2744300972421489,
 'roc_auc': 0.7628665569086446,
 'train_roc_auc': 0.7557983686938431,
 'pr_roc_auc': 0.23600742119957863}

In [9]:
# Random CV method
apps_cv_random = pd.read_csv("data/apps_cv_random.csv")
apps_holdout_random = pd.read_csv("data/apps_holdout_random.csv")
target_col = 'TARGET'
feature_cols = [col for col in apps_cv_random.columns if col not in 
                [target_col, 'SK_ID_CURR', 'fold', 'neighbors_target_mean_500' , 'AGE_INT', 'DAYS_BIRTH', 'CODE_GENDER_M', 'CODE_GENDER_XNA', 'NAME_FAMILY_STATUS_Previously Married', 'NAME_FAMILY_STATUS_Single']]
param_grid = {
    "C": [0.001, 0.01, 0.1, 1, 10, 100],
    "loss": ["hinge", "squared_hinge"]
}

wide_search_results = grid_search_SVC(apps_cv_random, feature_cols, target_col, param_grid)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


1/12 (  8.3% in 1.0 mins) | Best ROC-AUC: 0.7567 | Best Params: {'C': 0.001, 'loss': 'hinge'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


2/12 ( 16.7% in 8.6 mins) | Best ROC-AUC: 0.7567 | Best Params: {'C': 0.001, 'loss': 'hinge'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


...

10/12 ( 83.3% in 59.4 mins) | Best ROC-AUC: 0.7570 | Best Params: {'C': 0.1, 'loss': 'hinge'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


11/12 ( 91.7% in 66.6 mins) | Best ROC-AUC: 0.7570 | Best Params: {'C': 0.1, 'loss': 'hinge'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


12/12 (100.0% in 74.0 mins) | Best ROC-AUC: 0.7570 | Best Params: {'C': 0.1, 'loss': 'hinge'}


In [10]:
wide_search_results

,params,mean_roc_auc,mean_f1,mean_acc,mean_bal_acc,mean_prec,mean_rec
0,"{'C': 0.1, 'loss': 'hinge'}",0.756992,0.267886,0.699470,0.692082,0.166613,0.683271
1,"{'C': 0.01, 'loss': 'hinge'}",0.756830,0.267589,0.698770,0.691957,0.166350,0.683831
2,"{'C': 0.01, 'loss': 'squared_hinge'}",0.756820,0.267741,0.699295,0.691937,0.166506,0.683162
3,"{'C': 0.001, 'loss': 'hinge'}",0.756717,0.266931,0.697468,0.691550,0.165803,0.684491
4,"{'C': 0.001, 'loss': 'squared_hinge'}",0.756675,0.267401,0.698929,0.691575,0.166264,0.682805
5,"{'C': 0.1, 'loss': 'squared_hinge'}",0.756660,0.269067,0.703205,0.692107,0.167790,0.678874
6,"{'C': 1, 'loss': 'hinge'}",0.749659,0.266898,0.709272,0.685664,0.167446,0.657532
7,"{'C': 1, 'loss': 'squared_hinge'}",0.733219,0.278840,0.799061,0.654478,0.196618,0.482168
8,"{'C': 10, 'loss': 'squared_hinge'}",0.603407,0.135945,0.864149,0.534937,0.140985,0.142560
9,"{'C': 10, 'loss': 'hinge'}",0.586375,0.121858,0.871481,0.524952,0.141672,0.111985


In [11]:
powers = np.arange(-3, -1, 0.1)
grid_values = [10**x for x in powers]

param_grid = {
    "C": grid_values,
    "loss": ["hinge"]
}

search_results2 = grid_search_SVC(apps_cv_random, feature_cols, target_col, param_grid)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


1/20 (  5.0% in 0.9 mins) | Best ROC-AUC: 0.7567 | Best Params: {'C': 0.001, 'loss': 'hinge'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


2/20 ( 10.0% in 1.9 mins) | Best ROC-AUC: 0.7567 | Best Params: {'C': 0.0012589254117941675, 'loss': 'hinge'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


...

18/20 ( 90.0% in 45.8 mins) | Best ROC-AUC: 0.7569 | Best Params: {'C': 0.03981071705534985, 'loss': 'hinge'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


19/20 ( 95.0% in 51.2 mins) | Best ROC-AUC: 0.7569 | Best Params: {'C': 0.03981071705534985, 'loss': 'hinge'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


20/20 (100.0% in 56.9 mins) | Best ROC-AUC: 0.7569 | Best Params: {'C': 0.03981071705534985, 'loss': 'hinge'}


In [14]:
search_results2

,params,mean_roc_auc,mean_f1,mean_acc,mean_bal_acc,mean_prec,mean_rec
0,"{'C': 0.03981071705534985, 'loss': 'hinge'}",0.756918,0.267692,0.699063,0.691981,0.166447,0.683534
1,"{'C': 0.03162277660168389, 'loss': 'hinge'}",0.756883,0.267520,0.698864,0.691800,0.166324,0.683374
2,"{'C': 0.01995262314968885, 'loss': 'hinge'}",0.756877,0.267410,0.698783,0.691663,0.166251,0.683171
3,"{'C': 0.025118864315095874, 'loss': 'hinge'}",0.756861,0.267333,0.698669,0.691600,0.166191,0.683170
4,"{'C': 0.010000000000000021, 'loss': 'hinge'}",0.756849,0.267437,0.698669,0.691762,0.166251,0.683524
5,"{'C': 0.015848931924611172, 'loss': 'hinge'}",0.756845,0.267499,0.698742,0.691825,0.166296,0.683576
6,"{'C': 0.012589254117941701, 'loss': 'hinge'}",0.756843,0.267504,0.698750,0.691830,0.166300,0.683577
7,"{'C': 0.07943282347242846, 'loss': 'hinge'}",0.756837,0.267724,0.699784,0.691678,0.166562,0.682011
8,"{'C': 0.00501187233627273, 'loss': 'hinge'}",0.756833,0.267510,0.698673,0.691882,0.166292,0.683783
9,"{'C': 0.00794328234724283, 'loss': 'hinge'}",0.756830,0.267422,0.698648,0.691751,0.166239,0.683526


We found the best value of `C` to be 10^-1.4, with 10^-1.5 in a close second. Hinge loss was still better than squared hinge

In [15]:
# Multi-stratified CV method
apps_cv_multi = pd.read_csv("data/apps_cv_multi.csv")
apps_holdout_multi = pd.read_csv("data/apps_holdout_multi.csv")
target_col = 'TARGET'
feature_cols = [col for col in apps_cv_multi.columns if col not in 
                [target_col, 'SK_ID_CURR', 'fold', 'neighbors_target_mean_500' , 'AGE_INT', 'DAYS_BIRTH', 'CODE_GENDER_M', 'CODE_GENDER_XNA', 'NAME_FAMILY_STATUS_Previously Married', 'NAME_FAMILY_STATUS_Single']]
param_grid = {
    "C": [0.001, 0.01, 0.1, 1, 10, 100],
    "loss": ["hinge", "squared_hinge"]
}

wide_search_results = grid_search_SVC(apps_cv_multi, feature_cols, target_col, param_grid)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


1/12 (  8.3% in 0.9 mins) | Best ROC-AUC: 0.7559 | Best Params: {'C': 0.001, 'loss': 'hinge'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


2/12 ( 16.7% in 7.8 mins) | Best ROC-AUC: 0.7559 | Best Params: {'C': 0.001, 'loss': 'hinge'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


...

10/12 ( 83.3% in 56.9 mins) | Best ROC-AUC: 0.7562 | Best Params: {'C': 0.1, 'loss': 'hinge'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


11/12 ( 91.7% in 65.0 mins) | Best ROC-AUC: 0.7562 | Best Params: {'C': 0.1, 'loss': 'hinge'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


12/12 (100.0% in 72.8 mins) | Best ROC-AUC: 0.7562 | Best Params: {'C': 0.1, 'loss': 'hinge'}


In [16]:
wide_search_results

,params,mean_roc_auc,mean_f1,mean_acc,mean_bal_acc,mean_prec,mean_rec
0,"{'C': 0.1, 'loss': 'hinge'}",0.756183,0.267588,0.698647,0.690972,0.166460,0.681819
1,"{'C': 0.1, 'loss': 'squared_hinge'}",0.756166,0.268697,0.702357,0.690898,0.167599,0.677233
2,"{'C': 0.01, 'loss': 'squared_hinge'}",0.756120,0.267153,0.698687,0.690258,0.166220,0.680206
3,"{'C': 0.01, 'loss': 'hinge'}",0.755990,0.266854,0.697219,0.690494,0.165854,0.682474
4,"{'C': 0.001, 'loss': 'hinge'}",0.755873,0.266311,0.695774,0.690329,0.165355,0.683835
5,"{'C': 0.001, 'loss': 'squared_hinge'}",0.755843,0.266743,0.698053,0.689913,0.165903,0.680206
6,"{'C': 1, 'loss': 'hinge'}",0.751548,0.265204,0.698899,0.686995,0.165182,0.672799
7,"{'C': 1, 'loss': 'squared_hinge'}",0.733508,0.280579,0.802081,0.654125,0.199018,0.477675
8,"{'C': 10, 'loss': 'squared_hinge'}",0.629641,0.164940,0.852526,0.548622,0.155944,0.186196
9,"{'C': 100, 'loss': 'squared_hinge'}",0.589629,0.147128,0.846085,0.536706,0.132592,0.167749


In [17]:
powers = np.arange(-3, 0, 0.1)
grid_values = [10**x for x in powers]

param_grid = {
    "C": grid_values,
    "loss": ["hinge"]
}

search_results2 = grid_search_SVC(apps_cv_multi, feature_cols, target_col, param_grid)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the n

2/30 (  6.7% in 1.8 mins) | Best ROC-AUC: 0.7559 | Best Params: {'C': 0.0012589254117941675, 'loss': 'hinge'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


3/30 ( 10.0% in 2.8 mins) | Best ROC-AUC: 0.7559 | Best Params: {'C': 0.001584893192461114, 'loss': 'hinge'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the n

...

27/30 ( 90.0% in 92.2 mins) | Best ROC-AUC: 0.7563 | Best Params: {'C': 0.15848931924611206, 'loss': 'hinge'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the n

29/30 ( 96.7% in 103.9 mins) | Best ROC-AUC: 0.7563 | Best Params: {'C': 0.15848931924611206, 'loss': 'hinge'}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


30/30 (100.0% in 109.9 mins) | Best ROC-AUC: 0.7563 | Best Params: {'C': 0.15848931924611206, 'loss': 'hinge'}


In [18]:
search_results2

,params,mean_roc_auc,mean_f1,mean_acc,mean_bal_acc,mean_prec,mean_rec
0,"{'C': 0.15848931924611206, 'loss': 'hinge'}",0.756335,0.267729,0.698866,0.691068,0.166577,0.681768
1,"{'C': 0.1995262314968889, 'loss': 'hinge'}",0.756278,0.267526,0.698765,0.690806,0.166444,0.681315
2,"{'C': 0.10000000000000041, 'loss': 'hinge'}",0.756204,0.267138,0.698309,0.690420,0.166161,0.681013
3,"{'C': 0.06309573444801955, 'loss': 'hinge'}",0.756150,0.267289,0.697670,0.690969,0.166160,0.682978
4,"{'C': 0.025118864315095874, 'loss': 'hinge'}",0.756097,0.267236,0.697678,0.690882,0.166132,0.682776
5,"{'C': 0.03162277660168389, 'loss': 'hinge'}",0.756097,0.267076,0.697430,0.690747,0.166007,0.682776
6,"{'C': 0.0501187233627274, 'loss': 'hinge'}",0.756077,0.267218,0.697695,0.690845,0.166124,0.682676
7,"{'C': 0.07943282347242846, 'loss': 'hinge'}",0.756076,0.267162,0.697585,0.690808,0.166078,0.682726
8,"{'C': 0.03981071705534985, 'loss': 'hinge'}",0.756076,0.267215,0.697691,0.690843,0.166121,0.682676
9,"{'C': 0.12589254117941726, 'loss': 'hinge'}",0.756068,0.267503,0.698146,0.691067,0.166350,0.682625


We found the best value of `C` to be 10^-0.8, although the hyperparameter C appears to be quite unstable with regards to optimizing ROC-AUC. 10^-1.5 is 5th and 10^-1.4 is 8th